# Setup for model

- Instructed to use NASA's simupy-flight.
Check README for setup instructions.

In [1]:
# Instructed to use simupy.
# Here we can configure our Vehicle object.

from simupy_flight import Vehicle, Planet

print(help(Vehicle))

Help on class Vehicle in module simupy_flight:

class Vehicle(builtins.object)
 |  Vehicle(m, I_xx, I_yy, I_zz, I_xy, I_yz, I_xz, x_com, y_com, z_com, base_aero_coeffs, x_mrc, y_mrc, z_mrc, S_A, a_l, b_l, c_l, d_l, input_aero_coeffs=0.0, input_force_moment=0.0, input_aero_coeffs_idx=None, input_force_moment_idx=None, dim_additional_input=0)
 |  
 |  The Vehicle is a state-less dynamics block that provides a common calculation for
 |  the dynamics of a flight vehicle.
 |  
 |  The Vehicle vehicle model is parameterized based on the following components:
 |  
 |  Inertial properties: ``m, I_xx, I_yy, I_zz, I_xy, I_yz, I_xz, x_com, y_com, z_com``
 |      Inertia is expressed in body-fixed coordinates about the center of mass
 |      position (``x_com, y_com, z_com``). Center of mass position is relative to an
 |      arbitrary reference, such as the CAD origin.
 |  
 |  Aerodynamics model:
 |  
 |  ``base_aero_coeffs``:
 |      assumed to be a function of angles of attack and sideslip, an

In [3]:
# imports for planet setup, using print(help(Planet)), mirroring simupy-flight/nesc_test_cases/nesc_case01.py
from simupy_flight import (
    earth_J2_gravity,
    get_constant_winds,
    get_constant_atmosphere,
    Planetodetic,
    earth_equitorial_radius,
    earth_rotation_rate,
    earth_f,
)

In [4]:
# my blimp is designed to fly on Earth, for now:
planet = Planet(
    gravity=earth_J2_gravity,
    winds=get_constant_winds(),
    atmosphere=get_constant_atmosphere(),
    planetodetics=Planetodetic(
        a=earth_equitorial_radius,
        omega_p=earth_rotation_rate,
        f=earth_f,
    ),
)

### Output from Vehicle help:
class Vehicle(builtins.object)
 |  Vehicle(m, I_xx, I_yy, I_zz, I_xy, I_yz, I_xz, x_com, y_com, z_com, base_aero_coeffs, x_mrc, y_mrc, z_mrc, S_A, a_l, b_l, c_l, d_l, input_aero_coeffs=0.0, input_force_moment=0.0, input_aero_coeffs_idx=None, input_force_moment_idx=None, dim_additional_input=0)
 |  
 |  The Vehicle is a state-less dynamics block that provides a common calculation for
 |  the dynamics of a flight vehicle.
 |  
 |  The Vehicle vehicle model is parameterized based on the following components:
 |  
 |  Inertial properties: ``m, I_xx, I_yy, I_zz, I_xy, I_yz, I_xz, x_com, y_com, z_com``
 |      Inertia is expressed in body-fixed coordinates about the center of mass
 |      position (``x_com, y_com, z_com``). Center of mass position is relative to an
 |      arbitrary reference, such as the CAD origin.
 |  
 |  Aerodynamics model:
 |  
 |  ``base_aero_coeffs``:
 |      assumed to be a function of angles of attack and sideslip, and Mach and Reynolds
 |      number with signature ``base_aero_coeffs(alpha, beta, Ma, Re)`` and should
 |      return an array of coefficients in the following order: drag, sideforce, and
 |      lift (force coefficients expressed in wind coordinate system), static roll,
 |      pitch, and yaw moments (moment coefficients expressed in body-fixed
 |      Forward-Right-Down coordinate system), and dynamic or damping roll, pitch, and
 |      yaw moments (also moment coefficients expressed in body-fixed coordinate
 |      system). To translate an aerodynamic database that provides forces expressed in
 |      body-fixed coordinates, the transpose of the ``dynamics.body_to_wind_dcm(alpha,
 |      beta)`` direction cosine matrix can be used to transform into the wind
 |      coordinate system. See :func:`get_constant_aero`.
 |  ``x_mrc, y_mrc, z_mrc``:
 |      Moment reference center position, defined relative to the same arbitrary
 |      reference as the center of mass. Moment coefficients are assumed to be about
 |      this position.
 |  ``S_A``:
 |      the reference area
 |  ``a_l, b_l, c_l``:
 |      reference lengths for roll, pitch, and yaw
 |  ``d_l``
 |      reference length for Reynolds number calculation
 |  
 |  Damping coefficients are transformed with the static moment coefficients, which does
 |  not usually hold; to model damping coefficients, it is recommended to set the center
 |  of mass and moment reference center to the same location at the arbitrary reference
 |  (i.e. (0, 0, 0)).
 |  
 |  Additional, controlled input can be modeled:
 |  
 |  ``input_force_moment, input_force_moment_idx``:
 |      Non-aerodynamic controlled inputs (e.g. propulsion)
 |  ``input_aero_coeffs, input_aero_coeffs_idx``:
 |      Controlled aerodynamics
 |  
 |  Processing of ``input_aero_coeffs`` and ``input_force_moment`` parameterized models
 |  follow the same logic:
 |  
 |  - if ``None``: assume that a child class will over-write
 |  - if callable: use directly (so it should have the correct signature)
 |      - The ``input_aero_coeffs`` callback is assumed to take the flight condition
 |        arguments of the aerodynamics model, before any additional routed control
 |        inputs, and return aerodynamic coefficient increments.
 |  
 |      - The ``input_force_moment`` callback takes only routed control inputs and
 |        returns the translational forces and moments (about the center of mass) being
 |        modeled, such as for a propulsion model.
 |  - else: try to build a function that returns constant value(s)
 |  
 |  Attributes ``input_aero_coeffs_idx`` and ``input_force_moment_idx`` are used to
 |  route extra (control) inputs to the input aero and foce/moment functions
 |  respectively. Use ``None`` to route all inputs or use a list of integers to route
 |  particular inputs including an empty list for no routing. ``dim_additional_input``
 |  as an input argument to ``__init__`` is used to allocate extra control input
 |  channels in the block diagram.
 |  
 |  The **input** components are:
 |  
 |  ``[0:13] p_x, p_y, p_z, q_0, q_1, q_2, q_3, v_x, v_y, v_z, omega_X, omega_Y, omega_Z``
 |      The vehicle state components
 |  ``[13:16] lamda_D, phi_D, h``
 |      translational position of the vehicle center of mass in the planet-fixed frame
 |      expressed in planetodetic spherical position coordinates: longitude, latitude,
 |      and altitude
 |  ``[16:19] psi, theta, phi``
 |      Euler-angles relating the NED frame to the body-fixed frame: yaw, pitch, roll
 |  ``[19:22] rho, c_s, mu``
 |      density, speed of sound, and viscosity outputs of the atmosphere model as a
 |      function of time and position in planet-fixed frame
 |  ``[22:25] V_T, alpha, beta``
 |      true air speed, angle of attack, and angle of sideslip used for aerodynamics
 |      (includes wind)
 |  ``[25:28] p_B, q_B, r_B``
 |      angular velocity components of body relative to NED used for aerodynamic damping
 |      derivatives
 |  ``[28:31] V_N, V_E, V_D``
 |      velocity of the vehicle center of mass in the planet-fixed frame expressed in
 |      NED frame
 |  ``[31:34] W_N, W_E, W_D``
 |      winds acting on the vehicle expressed in NED frame
 |  
 |  The **output** components are:
 |  
 |  ``[0:3] A_X, A_Y, A_Z``
 |      translational acceleration of vehicle center of mass due to non-gravitational
 |      forces in the inertial coordinate system expressed in body-fixed
 |      Forward-Right-Down (FRD) coordinates, computed assuming constant mass and total
 |      non-gravitational forces due to aerodynamics (base model and extra aerodynamic
 |      coefficients input components) and the extra force input components.
 |  ``[3:6] alpha_X, alpha_Y, alpha_Z``
 |      angular acceleration of vehicle coordinate system in the inertial coordinate
 |      system expressed in body-fixed FRD coordinates, about the center of mass
 |      computed assuming constant inertia and total moments due to aerodynamics (base
 |      model and extra aerodynamic coefficients input components) and the extra moment
 |      input components.
 |  
 |  Additional class attributes:
 |  
 |  ``dim_state, dim_output, dim_input, num_events``
 |      Used for the SimuPy interface; must be updated for any sub-classes
 |  ``<variable_name>_idx``
 |      Provides a named index value for indexing the state or output data columns
 |  ``output_column_names``
 |      A list of strings of the output names, useful for constructing data structures
 |  ``output_column_names_latex``
 |      A list of strings of the LaTeX output names, useful for labeling plots
 |  
 |  Methods defined here:
 |  
 |  __init__(self, m, I_xx, I_yy, I_zz, I_xy, I_yz, I_xz, x_com, y_com, z_com, base_aero_coeffs, x_mrc, y_mrc, z_mrc, S_A, a_l, b_l, c_l, d_l, input_aero_coeffs=0.0, input_force_moment=0.0, input_aero_coeffs_idx=None, input_force_moment_idx=None, dim_additional_input=0)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  mrc_to_com_cpm(self)
 |      Construct a skew symmetric matrix that can perform the right-sided cross product
 |      of the position vector from the moment reference center to the center of mass,
 |      used for accounting for the translational force contribution to the moment.
 |  
 |  output_equation_function(self, t, u)
 |      Compute the dynamic outputs used for simulation.
 |  
 |  prepare_to_integrate(self, *args, **kwargs)
 |      SimuPy calls each Block's ``prepare_to_integrate`` function prior to simulation
 |      Returns the first output vector.
 |  
 |  tot_aero_forces_moments(self, qbar, Ma, Re, V_T, alpha, beta, p_B, q_B, r_B, *args)
 |      Helper function to compute the total aerodynamic forces and moments for a given
 |      flight  and vehicle condition. Used to evaluate the aerodynamic forces and
 |      moments in calculating the dynamics output.
 |      
 |      Parameters
 |      ----------
 |      qbar
 |          dynamic pressure in Pascals
 |      Mach
 |          Mach number (unitless)
 |      Re
 |          Reynolds number (unitless)
 |      V_T, alpha, beta
 |          true air speed, angle of attack, and angle of sideslip used for aerodynamics
 |          (includes wind)
 |      p_B, q_B, r_B
 |          angular velocity components of body relative to NED used for aerodynamic
 |          damping derivatives
 |      *args
 |          Captures any additional control inputs for the controlled aerodynamics model
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  Ma_arg_idx = 36
 |  
 |  Re_arg_idx = 37
 |  
 |  V_D_arg_idx = 31
 |  
 |  V_E_arg_idx = 30
 |  
 |  V_N_arg_idx = 29
 |  
 |  V_T_arg_idx = 23
 |  
 |  W_D_arg_idx = 34
 |  
 |  W_E_arg_idx = 33
 |  
 |  W_N_arg_idx = 32
 |  
 |  alpha_arg_idx = 24
 |  
 |  beta_arg_idx = 25
 |  
 |  c_s_arg_idx = 21
 |  
 |  dim_input = 34
 |  
 |  dim_output = 6
 |  
 |  dim_state = 0
 |  
 |  h_D_arg_idx = 16
 |  
 |  initial_condition = array([], dtype=float64)
 |  
 |  lamda_D_arg_idx = 14
 |  
 |  mu_arg_idx = 22
 |  
 |  num_events = 0
 |  
 |  omega_X_arg_idx = 11
 |  
 |  omega_Y_arg_idx = 12
 |  
 |  omega_Z_arg_idx = 13
 |  
 |  output_column_names = ['A_X', 'A_Y', 'A_Z', 'alpha_X', 'alpha_Y', 'alp...
 |  
 |  output_column_names_latex = ['$A_{X}$', '$A_{Y}$', '$A_{Z}$', r'$\alph...
 |  
 |  p_B_arg_idx = 26
 |  
 |  p_x_arg_idx = 1
 |  
 |  p_y_arg_idx = 2
 |  
 |  p_z_arg_idx = 3
 |  
 |  phi_D_arg_idx = 15
 |  
 |  phi_arg_idx = 19
 |  
 |  psi_arg_idx = 17
 |  
 |  q_0_arg_idx = 4
 |  
 |  q_1_arg_idx = 5
 |  
 |  q_2_arg_idx = 6
 |  
 |  q_3_arg_idx = 7
 |  
 |  q_B_arg_idx = 27
 |  
 |  qbar_arg_idx = 35
 |  
 |  r_B_arg_idx = 28
 |  
 |  rho_arg_idx = 20
 |  
 |  starargs_idx = 38
 |  
 |  t_arg_idx = 0
 |  
 |  theta_arg_idx = 18
 |  
 |  v_x_arg_idx = 8
 |  
 |  v_y_arg_idx = 9
 |  
 |  v_z_arg_idx = 10

In [ ]:
vehicle = Vehicle(
    m,
    I_xx,
    I_yy,
    I_zz,
    I_xy,
    I_yz,
    I_xz,
    x_com,
    y_com,
    z_com,  # inertia
    base_aero_coeffs,
    x_mrc,
    y_mrc,
    z_mrc,
    S_A,
    a_l,
    b_l,
    c_l,
    d_l,  # aero
    input_aero_coeffs,
    input_force_moment,  # extra callbacks for control modeling
    input_aero_coeffs_idx,
    input_force_moment_idx,  # routing for control callbacks
    dim_extra_input,  # total number of extra (control) inputs
)

planet = Planet(gravity, winds, atmosphere, planetodetics)